In [1]:
# Proper usage
md_content = """
## Current Assets
### Cash and Cash Equivalents
Currency in checking/savings accounts
Short-term Treasury bills (maturing <3 months)
Commercial paper from AAA-rated corporations
Money market funds with daily liquidity
Petty cash reserves for office expenses
Foreign currency holdings in major currencies
Undeposited checks from customers
Cash in transit between bank accounts
### Marketable Securities
Corporate bonds with <1yr maturity
Government agency securities
Certificates of deposit (CDs)
Bankers' acceptances
Commercial paper holdings
Treasury notes maturing within 12 months
Highly liquid ETF positions
### Accounts Receivable
Trade receivables from normal operations
Installment receivables from long-term contracts
Receivables from affiliated companies
Allowance for doubtful accounts calculation
Aging schedule analysis (30/60/90 days)
Credit memo adjustments
Factored receivables disclosure
Unbilled receivables from progress contracts

## Non-Current Assets
### Property, Plant & Equipment
Land acquisition costs (original purchase)
Building improvements capitalization
Machinery installation costs
Equipment depreciation schedules
Leasehold improvement amortization
Construction-in-progress accounts
Capitalized interest during construction
### Intangible Assets
Patent acquisition and amortization
Trademark registration/maintenance costs
Customer list valuations
Non-compete agreement valuations
Software development costs
Licensing agreements fair value
Goodwill impairment testing methodology
### Long-Term Investments
Held-to-maturity securities portfolio
Equity method investment accounting
Real estate held for appreciation
Venture capital fund investments
Convertible debt instruments
Restricted stock holdings
Investments in subsidiaries

## Current Liabilities
### Accounts Payable
Trade payables to suppliers
Accrued purchases for goods received
Third-party processor withholdings
Construction retainage payable
Dividends declared but unpaid
Customer deposits/advance payments
Escheat liability estimates
### Short-Term Debt
Commercial paper outstanding
Revolving credit facility draws
Current portion of long-term debt
Bank overdraft facilities used
Short-term lease liabilities
Vendor financing arrangements
Convertible debt equity component

## Long-Term Liabilities
### Bonds Payable
Corporate bond issuance at premium/discount
Debenture conversion features
Sinking fund requirements
Unamortized bond issuance costs
Fair value hedge adjustments
Callable bond provisions
Convertible bond accounting
### Pension Liabilities
Defined benefit obligation calculations
Actuarial gains/losses recognition
Plan asset valuations
Curtailment/settlement accounting
Multi-employer plan disclosures
Post-employment benefits accrual
Termination benefit provisions

## Shareholders' Equity
### Common Stock
Par value per share disclosure
Authorized shares vs outstanding
Treasury stock accounting method
Stock split adjustments
Stock option pool reserves
Restricted stock unit accruals
Dividend reinvestment plan shares
### Retained Earnings
Prior period adjustments
Dividend declaration accounting
ESOP allocation impacts
Foreign currency translation adjustments
Hedging reserve balances
Revaluation surplus accounts
Accumulated other comprehensive income
"""


In [15]:
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor
from dataclasses import dataclass
from typing import List, Tuple
import textwrap
import logging

logging.basicConfig(level=logging.INFO)

@dataclass
class FinancialItem:
    accounting_type: str
    account_title: str
    descriptions: List[str]
    continued: bool = False
    remaining_lines: List[str] = None  # Track partially filled descriptions

class PowerPointGenerator:
    def __init__(self, template_path: str):
        self.prs = Presentation(template_path)
        self._validate_template()
        self.current_slide_index = 0
        self.current_section = 'c'
        self.ROWS_PER_SECTION = 28  # Updated row count
        self.CHARS_PER_ROW = 50     # Updated character count
        self.prev_item = None
        self.partial_item = None
        self.BULLET_CHAR = chr(0x25A0) + ' '  # Unicode black square ■
        self.DARK_GREY = RGBColor(169, 169, 169)  # #A9A9A9
        
    def _apply_paragraph_formatting(self, paragraph):
        """Apply uniform paragraph formatting"""
        paragraph_format = paragraph.paragraph_format
        paragraph_format.alignment = PP_ALIGN.LEFT
        paragraph_format.left_indent = Inches(0.21)
        paragraph_format.first_line_indent = Inches(-0.19)  # Hanging indent
        paragraph_format.space_before = Pt(6.06)
        paragraph_format.line_spacing = 1.0  # Single spacing

    def _validate_template(self):
        if len(self.prs.slides) < 4:
            logging.warning("Template has fewer than 4 slides, content may be truncated")

    def _calculate_chunk_size(self, items: List[FinancialItem]) -> Tuple[List[FinancialItem], FinancialItem]:
        """Calculate chunk with partial item handling"""
        lines_used = 0
        capacity = []
        
        for item in items:
            # Handle partial items from previous section
            if self.partial_item:
                item = self.partial_item
                self.partial_item = None

            # Calculate lines needed including remaining lines
            item_lines = 2  # Type + title
            if item.remaining_lines:
                item_lines += len(item.remaining_lines)
            else:
                item_lines += len(item.descriptions)
            
            if lines_used + item_lines > self.ROWS_PER_SECTION:
                # Split descriptions if possible
                if lines_used < self.ROWS_PER_SECTION:
                    available_lines = self.ROWS_PER_SECTION - lines_used - 2  # Reserve for type/title
                    if available_lines > 0:
                        partial_desc = (item.remaining_lines or item.descriptions)[:available_lines]
                        remaining_desc = (item.remaining_lines or item.descriptions)[available_lines:]
                        
                        # Create partial item for next section
                        self.partial_item = FinancialItem(
                            item.accounting_type,
                            item.account_title,
                            [],
                            continued=True,
                            remaining_lines=remaining_desc
                        )
                        
                        # Add partial item to current chunk
                        capacity.append(FinancialItem(
                            item.accounting_type,
                            item.account_title,
                            partial_desc,
                            item.continued
                        ))
                        lines_used += 2 + len(partial_desc)
                break
                
            lines_used += item_lines
            capacity.append(item)
            self.partial_item = None  # Reset partial after full consumption
            
        return capacity, self.partial_item

    def parse_markdown(self, md_content: str) -> List[FinancialItem]:
        """Convert markdown to structured financial items"""
        items = []
        current_type = ""
        current_title = ""
        current_descs = []
        
        for line in md_content.strip().split('\n'):
            line = line.strip()
            if not line:
                continue
                
            if line.startswith('## '):
                if current_type or current_title or current_descs:
                    items.append(FinancialItem(current_type, current_title, current_descs))
                current_type = line[3:].strip()
                current_title = ""
                current_descs = []
            elif line.startswith('### '):
                if current_title or current_descs:
                    items.append(FinancialItem(current_type, current_title, current_descs))
                current_title = line[4:].strip()
                current_descs = []
            else:
                current_descs.append(line)
                
        if current_type or current_title or current_descs:
            items.append(FinancialItem(current_type, current_title, current_descs))
            
        return items

    def _wrap_text(self, text: str) -> List[str]:
        return textwrap.wrap(text, width=self.CHARS_PER_ROW, break_long_words=True)

    def _calculate_item_lines(self, item: FinancialItem) -> int:
        """Calculate total lines required for an item"""
        wrapped_type = self._wrap_text(item.accounting_type)
        wrapped_title = self._wrap_text(item.account_title)
        wrapped_descs = [self._wrap_text(desc) for desc in item.descriptions]
        return len(wrapped_type) + len(wrapped_title) + sum(len(desc) for desc in wrapped_descs)

    def _get_section_shape(self, slide, section: str):
        """Retrieve shape based on section and slide position"""
        if self.current_slide_index == 0:
            return next((s for s in slide.shapes if s.name == "textMainBullets"), None)
        
        section_map = {'b': 'L', 'c': 'R'}
        target_name = f"textMainBullets_{section_map[section]}"
        return next((s for s in slide.shapes if s.name == target_name), None)

    def _populate_section(self, shape, items: List[FinancialItem]):
        """Fill shape with formatted content while preserving original layout"""
        if not shape:
            return

        tf = shape.text_frame
        tf.clear()
        tf.word_wrap = True
        current_lines = 0
        
        for item in items:
            # Layer 1: Accounting Type
            paragraph = tf.add_paragraph()
            run = paragraph.add_run()  # Create empty run first
            run.text = f"{item.accounting_type} (continued)" if item.continued else item.accounting_type
            run.font.bold = True
            run.font.size = Pt(9)
            run.font.color.rgb = RGBColor(0x00, 0x32, 0x96)  # Dark Blue

            # Layer 2: Account Title 
            paragraph = tf.add_paragraph()
            run = paragraph.add_run()  # Create empty run
            run.text = f"{item.account_title} (continued)" if item.continued else item.account_title
            run.font.italic = True
            run.font.size = Pt(9)

            # Layer 3: Descriptions with bullets
            for desc in item.descriptions:
                paragraph = tf.add_paragraph()
                run = paragraph.add_run()  # Create empty run
                run.text = f"{self.BULLET_CHAR}{desc}"
                run.font.size = Pt(9)

    def generate(self, md_content: str, output_path: str):
        """Enhanced generation with partial item tracking"""
        items = self.parse_markdown(md_content)
        item_ptr = 0
        
        while item_ptr < len(items) or self.partial_item:
            if self.current_slide_index >= len(self.prs.slides):
                logging.warning("Insufficient slides, content truncated")
                break

            slide = self.prs.slides[self.current_slide_index]
            section_shape = self._get_section_shape(slide, self.current_section)
            
            if not section_shape:
                self._advance_section()
                continue

            # Combine remaining items with partial item
            processing_items = []
            if self.partial_item:
                processing_items.append(self.partial_item)
                self.partial_item = None
            processing_items.extend(items[item_ptr:])
            
            chunk, partial = self._calculate_chunk_size(processing_items)
            self.partial_item = partial
            
            if chunk:
                self._populate_section(section_shape, chunk)
                item_ptr += len([i for i in chunk if i not in processing_items])  # Track consumed items
                self._add_continuation_marks(chunk)
            
            self._advance_section()

        self.prs.save(output_path)
        logging.info("Generation completed with improved splitting")
    
    def _add_continuation_marks(self, items: List[FinancialItem]):
        """Mark items that continue across sections"""
        if not items:
            return
        
        # Track previous item across calls
        if self.prev_item:
            if items[0].accounting_type == self.prev_item.accounting_type:
                items[0].continued = True
                if items[0].account_title == self.prev_item.account_title:
                    items[0].continued = True
        
        # Update previous item tracking
        self.prev_item = items[-1] if items else None

    def _advance_section(self):
        """Move to next section following specified order"""
        if self.current_slide_index == 0:
            self.current_slide_index += 1
            self.current_section = 'b'
        else:
            if self.current_section == 'b':
                self.current_section = 'c'
            else:
                self.current_slide_index += 1
                self.current_section = 'b'

# Usage
if __name__ == "__main__":
    generator = PowerPointGenerator("template.pptx")
    
    try:
        generator.generate(md_content, "financial_report.pptx")
    except Exception as e:
        logging.error(f"Generation failed: {str(e)}")


INFO:root:Generation completed with improved splitting
